In [4]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x :.2f}')
from get_dataframes import GetDataframes
import hjson

import matplotlib.pyplot as plt
import explo_data_analysis.eda_movies as eda

with open("config.hjson") as fp:
    config = hjson.load(fp)


- Importation de la dataframe contenant seulement les réalisateurs.
- Cette dataframe a déjà été nettoyé comme indiqué précédemment.

In [6]:
datas = GetDataframes(config)
link = "directors_movies"
df = datas.get_dataframes(
    link,
    True
)

2023-11-10 14:49:06 INFO     TSV files already exist.
2023-11-10 14:49:06 INFO     Parquet loaded ! Importing directors_movies...
2023-11-10 14:49:06 INFO     Dataframe directors_movies ready to use!


- On veut afficher quels réalisateurs ont réalisés le plus de films pour chaque décénnie.

In [7]:
grouped_df = df.groupby(
    ['cuts', 'person_name'],
            observed=True
).size().reset_index(name='total_film_directors').sort_values(by="total_film_directors")

top_directors_decennie = grouped_df.groupby(
    'cuts',
    observed=True
).apply(lambda x: x.nlargest(1, 'total_film_directors')).reset_index(drop=True)

decennies = top_directors_decennie['cuts']
noms_directors = top_directors_decennie['person_name']
nombre_films = top_directors_decennie['total_film_directors']

In [8]:
fig = go.Figure(data=[
    go.Bar(
        x=nombre_films,
        y=decennies,
        orientation='h',
        marker=dict(
            color='#66cdaa',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=noms_directors,
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Réalisateur N°1 par Décennie et Nombre de Films réalisés',
    xaxis_title='Nombre de Films réalisés',
    yaxis_title='Décennie'
)

fig.show()

- On veut afficher quels réalisateurs ont réalisés le plus de films pour chaque genre.

In [9]:
directors_by_genre = df.explode('titre_genres').groupby(['person_name', 'titre_genres']).size().reset_index(name='count')
top_directors_by_genre = directors_by_genre.sort_values('count', ascending=False).drop_duplicates('titre_genres')[:10][::-1]

fig = go.Figure(data=[
    go.Bar(
        x=top_directors_by_genre['count'],
        y=top_directors_by_genre['titre_genres'],
        orientation='h',
        marker=dict(
            color='#006b3c',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_directors_by_genre['person_name'],
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='white'
        )
    )
])

fig.update_layout(
    title='Réalisateurs les plus fréquemment associés aux top 10 des genres',
    xaxis_title='Nombre de Films',
    yaxis_title='Genres'
)

fig.show()

- On veut afficher quels réalistaeurs ont réalisateurs le plus de films au total.

In [10]:
directors_film_count = df.groupby('person_name').size().reset_index(name='film_count')

top_directors_film_count = directors_film_count.sort_values('film_count', ascending=False).head(10)[::-1]

fig = go.Figure(data=[
    go.Bar(
        x=top_directors_film_count['film_count'],
        y=top_directors_film_count['person_name'],
        orientation='h',
        marker=dict(
            color='#daa520',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_directors_film_count['film_count'],
        textposition='inside',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Nombre de films réalisés par réalisateurs',
    xaxis_title='Nombre de Films',
    yaxis_title='Genres'
)

fig.show()

- On veut afficher les réalisateurs ainsi que le nombre de vote total qu'on reçu les films qu'ils ont réalisés.

In [11]:
directors_by_votes = df.groupby('person_name')['rating_votes'].sum().reset_index()

top_directors_by_votes = directors_by_votes.sort_values('rating_votes', ascending=False).head(10)[::-1]

fig = go.Figure(data=[
    go.Bar(
        x=top_directors_by_votes['rating_votes'],
        y=top_directors_by_votes['person_name'],
        orientation='h',
        marker=dict(
            color='#daa520',
            line=dict(
                color='black',
                width=1
            )
        ),
        text=top_directors_by_votes['rating_votes'],
        textposition='auto',
        width=1,
        textfont=dict(
            size=14,
            color='black'
        )
    )
])

fig.update_layout(
    title='Top 10 des réalisateurs avec les films ayant eu le plus de votes',
    xaxis_title='Réalisateurs',
    yaxis_title='Total des votes'
)

fig.show()
